# Crypto Currency Data From CoinGecko API

## Packages

In [ ]:
%pip install ipython-sql prettytable

## Imported Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import prettytable
prettytable.DEFAULT = 'DEFAULT'
import sqlite3

## Log Progress Fuction

In [ ]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./log_progress","a") as f:
        f.write(f"{timestamp}:{message}" + '\n')

## Extract Fuction

This function retrieves cryptocurrency data from the CoinGecko API and saves it to a pandas DataFrame. The returned DataFrame is formatted for further analysis or processing.

Specifically, the function extracts information about the top 100 cryptocurrencies, including their ID, name, current price in USD, and market capitalization in USD.

In [ ]:
def extract(url):    # sourcery skip: raise-specific-error
    ''' This function aims to extract the required
    information from the CoinGecko and save it to a data frame. The
    function returns the data frame for further processing. '''
    params = {  
               'vs_currency': 'USD'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")
    data = response.json()
    df = pd.DataFrame(data)
    columns = ['id', 'name', 'current_price', 'market_cap']
    return df[columns]

## Transform the current price and market cap to EUR and GBP

This function accesses a CSV file containing exchange rate information and adds three new columns to the DataFrame. These columns represent the transformed values of the 'Market Cap' and 'Current Price' columns into their respective currencies.

The function utilizes the `exchange_rate.csv` file to obtain the current exchange rates for USD to EUR and USD to GBP


In [ ]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Get the exchange rate from the csv file
    exchangerate_df = pd.read_csv(csv_path)
    # Transform the exchange rate in the data frame to a dictionary, in order manipulate it.
    exchange_rate = exchangerate_df.set_index('Currency').to_dict()['Rate']

    # Added new columns
    df['current_price_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['current_price']]
    df['current_price_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['current_price']]
    df['market_cap_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['market_cap']]
    df['market_cap_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['market_cap']]
    return df

## Load Fuctions

The firta function saves the final data frame as a CSV file in
the provided path. Function returns nothing.

The second fuction save saves the final data frame to a database
table with the provided name. Function returns nothing.

Additionally, we include the function `connection_to_database`, which establishes the connection to the database

In [ ]:
def load_to_csv(df,new_path):
    df.to_csv(new_path)

def conection_to_database(database_name):
    try:
        db_connection = sqlite3.connect(database_name)
    except sqlite3.OperationalError as e:
        raise e
    else:
        print("connected")
    return db_connection

def load_to_db(df,sql_connection,table_name):
        df.to_sql(table_name,sql_connection, if_exists='replace', index=False)


## ETL Procces

Now that we have all the required functions for this process, we can create a DataFrame based on the data extracted from the CoinGecko API, save it to a CSV file, and load it into a database for subsequent analysis.

### Required variables

In [ ]:
api_url = "https://api.coingecko.com/api/v3/coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st"
exchange_rate_csv = './exchange_rate.csv'
data_csv_path='./Crypto_Data.csv'
db_name='CryptoData.db'
table_name='Crypto_Data'
log_progress("Variables are define. Intiating ETL process")

Extract data from the api

In [ ]:
df = extract(api_url)
df
log_progress("Extracted crypto data from CoinGecko API")

Transforming Data as required. In this case we are going to transform the market cap and current price, which is in USD to EUR and GBP

In [ ]:
df = transform(df, exchange_rate_csv)
df

## Loading data

Now we are saving the data to a new CSV file and loading it into the database.

In [ ]:
load_to_csv(df,data_csv_path)

This generates a new CSV file in the current working directory containing cryptocurrency data extracted from the CoinGecko API. Next step is going to be load the data in to a Database.

In [ ]:
conn = conection_to_database(db_name)
# In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor.
curs = conn.cursor()
load_to_db(df,conn,table_name)

## Querying

Now that we have establish the connection with the database, we stablish a connection between SQL magic module and the database CrytoData.db, in order to run queries in jupyter notebook

In [ ]:
%load_ext sql
%sql sqlite:///CryptoData.db

### Running Queries

Check if the table exist

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type='table'

Check the number of rows

In [ ]:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('Crypto_Data')

Check name of the columns

In [ ]:
%sql SELECT name,type from PRAGMA_TABLE_INFO('Crypto_Data')

List of the total coins

In [ ]:
%sql SELECT count(*) FROM Crypto_Data

Lets list 10 coins with a current_price less than 1 USD

In [ ]:
%sql SELECT name, current_price FROM Crypto_Data WHERE current_price < 1 LIMIT 10 

List all the coins that have the word 'Coin' in his name

In [ ]:
%sql SELECT name FROM Crypto_Data WHERE name LIKE '%coin%'

List last 10 Coins by Market Cap

In [ ]:
%sql SELECT * FROM crypto_data ORDER BY market_cap LIMIT 10

List Average Current Price of last 10 coins

In [ ]:
%sql SELECT AVG(current_price) as AVERAGE_PRICE_10 FROM( SELECT current_price FROM crypto_data ORDER BY market_cap LIMIT 10)